In [12]:
import pandas as pd 
import glob
import cv2 as cv
import random
import os

import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
import PIL.ImageOps    

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.utils
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
import torch.nn.functional as F


from sklearn.metrics import f1_score, accuracy_score


from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler
from tqdm.auto import tqdm
import math
from sklearn.model_selection import train_test_split


# Make Data Frame

In [13]:
df = pd.read_csv('review_lp31.csv')
df = df.rename(columns={'stars':'label','text':'text' })
small_df = df[0:10000]
small_df['label'] = small_df['label'] - 1

/var/tmp/ipykernel_6544/2050621005.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_df['label'] = small_df['label'] - 1


### Seperate train, validset

In [14]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(small_df , test_size=0.2, random_state=42)

In [15]:
from datasets import Dataset

small_train_dataset = Dataset.from_pandas(train_df)
small_eval_dataset = Dataset.from_pandas(test_df)
small_train_dataset[100]

{'label': 0,
 'text': "make an appointment for a sport physical for 3:30pm . it 's be now 5:30pm and we have yet to see a doctor . we come here last year for the same physical and i be impress at how thorough it be but if you have to wait over 2 hour with an appointment , there be clearly well option out there . the staff and doctor be very good but we wo n't be back again with that kind of wait .",
 '__index_level_0__': 6998}

### Tokenize

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


small_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
small_eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)

# Train Model

In [19]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [20]:
CFG = {
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':8,
    'SEED':41
}

In [21]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = 'a'
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [ ]:
class TransformerkDataset(Dataset):
    def __init__(self,data):
        self.data= data
        
    def __getitem__(self,index):
        data = self.data[index]
    
        return data
    
    def __len__(self):
        return len(self.data)

In [ ]:
train_dataset = SiameseNetworkDataset(train["review_img_path"].values, train["product_img_path"].values, train["label"].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)